

Narrative structure of models
------------

**Three examples**

Before we start, let's take a moment to talk about 2 articles that make use of computation in an advanced way and the different techniques used to tell a story from data and models.

>[Spies in the Skies](https://www.buzzfeed.com/peteraldhous/spies-in-the-skies) (Machine Learning mostly hidden)
<br><br>
*Each weekday, dozens of U.S. government aircraft take to the skies and slowly circle over American cities. Piloted by agents of the FBI and the Department of Homeland Security (DHS), the planes are fitted with high-resolution video cameras, often working with “augmented reality” software that can superimpose onto the video images everything from street and business names to the owners of individual homes. At least a few planes have carried devices that can track the cell phones of people below. Most of the aircraft are small, flying a mile or so above ground, and many use exhaust mufflers to mute their engines — making them hard to detect by the people they’re spying on.*
<br><br>
[Surgeon Scorecard](https://projects.propublica.org/surgeons/) (The story is the model)
<br><br>
*Guided by experts, ProPublica calculated death and complication rates for surgeons performing one of eight elective procedures in Medicare, carefully adjusting for differences in patient health, age and hospital quality. Use this database to know more about a surgeon before your operation.*
<br><br>
[Trump's Reddit](https://fivethirtyeight.com/features/dissecting-trumps-most-rabid-online-following/) (The model is the story)
<br><br>
*President Donald Trump’s administration, in its turbulent first months, has drawn fire from both the left and the right, including the ACLU, government ethics accountability groups and former Bush administration officials. But one group has shown nothing but unbridled enthusiasm for the president’s actions thus far: the over 380,000 members of r/The_Donald, one of the thousands of comment boards on Reddit, the fifth-most-popular website in the U.S.*

**Learning**

We are going to consider a general purpose learning machine. It was developed in large part by Leo Breiman -- he's written about the "two cultures" in modeling. A **decision tree** is statistical model that is at one time a useful visualization as well as a prediction engine.  

The data comes from the election in 2008, and specifically during the primaries. It was developed by Amanda Cox, an outstanding visual journalist at The New York Times.
<br>

![tree](https://static01.nyt.com/images/2008/04/16/us/0416-nat-subOBAMA.jpg)

Have a look. The tree is like a game of 20 questions. If you want to know how a county will vote, answer a few questions. What is the racial makeup? What about the HS graduation rate? Where in the country is the county? 

The questions are in the form of a *tree* (computer scientists grow their trees upside down). As you answer questions, you eventually end up at a leaf and a decision is made. Note that it's not perfect -- there is some uncertainty in the decision. 

To show that this isn't a one-off, ProPublica had a lovely project called the [Message Machine](https://projects.propublica.org/emails/) where they looked email messages sent out by [political campaigns](https://www.propublica.org/special/message-machine-you-probably-dont-know-janet) and [reverse engineered the logic that generated them.](https://www.propublica.org/article/message-machine-starts-providing-answers)

Let's load up the data and see how such a thing comes into existence.

In [ ]:
library(rpart)
library(dplyr)

In [ ]:
primary = read.csv("https://github.com/computationaljournalism/columbia2018/raw/master/data/primary.csv",as.is=TRUE)

head(primary)
dim(primary)

In [ ]:
names(primary)

At the point this graphic was published Montana, Oregon, South Dakota, Indiana, Kentucky, West Virginia, Pennsylvania, and South Carolina had yet to have their primaries. In all, there should be 2,261 counties for analysis. Here's where NY state had landed.

In [ ]:
select(filter(primary,state_postal=="NY"),winner,county_name)

And here is how the various counties across the country voted.

In [ ]:
table(primary$winner)

**Relationships between predictors and our outcome**

Now, given all these potential predictor variables, which ones "explain" county-level voting patterns? Suppose, for example, we start simply, and consider whether or not a majority of the county voted for Bush in 2004. 

In [ ]:
table(primary$winner,primary$pres04winner)

In [ ]:
290/(290+163)

In [ ]:
1047/(1047+740)

Therefore, Obama won about 64% of those counties not voting for Bush in 04, while Clinton won about 59% of those counties that did vote for Bush in 04. Now, consider the **simple prediction rule**: If a county voted for Bush in 04, we’ll say that they will vote for Clinton in the primary, while if a county was mostly in favor of Kerry in 04, we’ll assign the win to Obama. 

Of course this rule isn’t perfect; by applying it, we would make 163 + 740 =
903 mistakes (out of 2,261 counties, or about 40% error); we refer to these mistakes as having been "misclassified" by our simple rule. So the question becomes, can we do any better? What do you think?

In [ ]:
(163 + 740)

In [ ]:
(163 + 740)/2261

There might be better indicators of Obama’s success besides the vote in 2004 -- but how do we find them? The decision tree encodes a large search across the complete data set for predictors that are informative. 

Consider the top of the tree, "the root". Decision trees work by repeatedly splitting the data into two parts; the root or first "split" is the single division of the data into two pieces that produces the lowest misclassification error (well, it's a little more complicated, but this will do).

To investigate this a little further, let’s consider the predictor that represents the
percentage of a county that is African American; we now choose a breakpoint that divides the data into two pieces (those counties with a greater percentage of African Americans and those with a smaller percentage). We then form a table (as we did for counties that went for Bush or Kerry in 2004) and count the misclassification rate.

Suppose we take 10% as the cutoff. We get the following table. How many errors do we make? What is the misclassification rate?

In [ ]:
table(primary$winner,primary$black06pct > 0.1)

In [ ]:
# Error rate?


The 20% figure at the root of the tree was obtained by finding the magic point that minimizes the misclassification errors. In fact, the search was conducted over all the variables in the data set and all the possible splits; and this choice produced the smallest error.

Once this node has been chosen, we work our way down the tree, conducting the same search but on the specified subsets of the data, at each step attempting to minimize our errors. 

For quantitative variables, we have a choice of split points, and so the algorithm goes through a search. Here we vary a cutoff for the percentage of African Americans living in a county from essentially 0 to 0.5. For each choice of cutoff, we look at the associated error. 

What do you see?

In [ ]:
fractions = (1:500)/1000

n = length(fractions)
error = rep(0,n)

for(i in 1:n){
    
    # right branch
    tmp = select(filter(primary,black06pct > fractions[i]),winner)
    error.right = min(table(tmp))
    
    # left branch
    tmp = select(filter(primary,black06pct <= fractions[i]),winner)
    error.left = min(table(tmp))
    
    error[i] = error.left+error.right
}

In [ ]:
options(repr.plot.width=8, repr.plot.height=5)

plot(fractions,error,xlab="percentage black",ylab="misclassification error",type="l")

In [ ]:
range(primary$Bush04,na.rm=TRUE)

In [ ]:
fractions = (100:900)/1000

n = length(fractions)
error = rep(0,n)

for(i in 1:n){
    
    # right branch
    tmp = select(filter(primary,Bush04 > fractions[i]),winner)
    error.right = min(table(tmp))
    
    # left branch
    tmp = select(filter(primary,Bush04 <= fractions[i]),winner)
    error.left = min(table(tmp))
    
    error[i] = error.left+error.right
}

In [ ]:
plot(fractions,error,xlab="Bush margin",ylab="misclassification error",type="l")

This is essentially what the decision tree process does. It's a *greedy algorithm*, meaning at each step, it is looking for the best way to divide the data into two pieces. There are things we can do to speed it up computationally, and there are issues with really trying to minimize misclassification rates, but this is essentially the approach.

Stop and think what this simple process has produced for us; we have a very intuitive structure (something akin to the game 20 questions) that makes evident "important" variables that help "explain" voting patterns. This kind of tool lives somewhere between data analysis and modeling; it is technically a model all by itself (making predictions) but is often used as a way to identify important predictors for another stage of model.

This decision tree is part of a large class of methods called CART for Classification and Regression Trees and was developed in the 1980s as part of a move to deal with bigger and meaner data sets.

Here is the implementation in R.

In [ ]:
library(rpart)

In [ ]:
fit = rpart(winner~region+pres04margin+black06pct+hisp06pct+white06pct+
            growth+pct_less_30k+pct_more_100k+pct_hs_grad+pct_homeowner+POP05_SQMI,data=primary)

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)
par(xpd=TRUE)

plot(fit)
text(fit,use.n=TRUE,cex=0.8)

Let’s think a bit more about the tree-growing process; with each split, we cut
our data at the node into two pieces so that the “sample size” at each of the child nodes is lower than its parents. We then represent the data in the leaves with a simple model; for our 0/1 data (Obama or Clinton), we classify leaves according to majority vote.

In principle we can grow trees until there’s a single entry in each node -- What
might be the problem with that? How do we decide to stop splitting? When we
run out of data? 

To grow a bigger tree, we can reduce "cp", the complexity parameter. Any split that does not decrease the overall lack of fit by a factor of cp is not attempted by our algorithm, so making it smaller will produce a larger tree.

In [ ]:
fit = rpart(winner~region+pres04margin+black06pct+hisp06pct+white06pct+
            growth+pct_less_30k+pct_more_100k+pct_hs_grad+pct_homeowner+POP05_SQMI,data=primary,cp=0.00001)

In [ ]:
plot(fit)

In the R displays above, the heights of the branches correspond to the error in represented by the model. So, the at the root, we are dealing with all the counties -- 1210 went for Clinton
and 1031 for Obama -- therefore since Clinton won more counties, we would
predict all future counties for Clinton, making 1030 mistakes.

The first split on the percentage of the county that is
African American brought us down to 700 errors, a big drop; the next division
based on education is another big drop, about half the size. As we continue to refine the tree, however, the improvements diminish.

In the mid-1980s a fair bit of theoretical and methodological work was devoted
to understanding the behavior of this kind of algorithm; we are using it as a bit
of data analysis (how does a “response” relate to the potential “explanatory”
variables?) but it can also be used as a tool for making predictions.

The R command rpart (for recursive partitioning) employs a “pruning”
algorithm that divides the original data into several parts; iteratively leaving one
part out, building a tree and evaluating it on the part that was left out -- this is
called cross validation.

In [ ]:
options(repr.plot.width=8, repr.plot.height=5)

plotcp(fit)

In [ ]:
printcp(fit)

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)

plot(prune(fit,0.01))

A lovely graphical introduciton to decision trees is [given here.](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)

**Logistic regression**

Decision trees are a relatively late addition to the statistian's toolbox. The narrative structure of a table as we have seen is also possible when our response is not normally distributed. So-called logistic regression uses "binomial" responses as the model. Think of a coin toss where the probability of turning up heads depends on a set of variables. So a county's primary is decided by a coin toss, the properties of that coin depending on demographics and so on.

In [ ]:
fit = glm(factor(winner)~region+pres04margin+black06pct+hisp06pct+white06pct+
          growth+pct_less_30k+pct_more_100k+pct_hs_grad+pct_homeowner+POP05_SQMI,
          data=primary,family=binomial)

In [ ]:
summary(fit)

In [ ]:
primary = mutate(primary,biwinner = winner=="obama")
head(primary)

In [ ]:
options(repr.plot.width=8, repr.plot.height=5)

plot(biwinner~Bush04,data=primary,pch="|")

In [ ]:
plot(biwinner~Bush04,data=primary,pch="|")
abline(lm(biwinner~Bush04,data=primary))

In [ ]:
plot(biwinner~black06pct,data=primary,pch="|")
abline(lm(biwinner~black06pct,data=primary))

In [ ]:
plot(biwinner~black06pct,data=primary,pch="|",col=ifelse(biwinner,"cyan","magenta"))
abline(lm(biwinner~black06pct,data=primary))

# ignore as slightly tedious "behind the scenes"
points(c(0.1,0.3,0.5,0.7,0.9),sapply(split(primary$biwinner,cut(primary$black06pct,seq(0,1,len=6))),mean,na.rm=TRUE),pch=19)
abline(v=c(0,0.2,0.4,0.6,0.8,1.0),lty=2)

In [ ]:
plot(biwinner~Bush04,data=primary,pch="|",col=ifelse(biwinner,"cyan","magenta"))
abline(lm(biwinner~Bush04,data=primary))

# ignore as slightly tedious "behind the scenes"
points(c(0.1,0.3,0.5,0.7,0.9),sapply(split(primary$biwinner,cut(primary$Bush04,seq(0,1,len=6))),mean,na.rm=TRUE),pch=19)
abline(v=c(0,0.2,0.4,0.6,0.8,1.0),lty=2)

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)

plot(black06pct~pct_hs_grad,data=primary,col=ifelse(biwinner,"cyan","magenta"),pch=ifelse(biwinner,"1","0"))

In [ ]:
fit = glm(factor(winner)~region+pres04margin+black06pct+hisp06pct+white06pct+
          growth+pct_less_30k+pct_more_100k+pct_hs_grad+pct_homeowner+POP05_SQMI,
          data=primary,family=binomial)

In [ ]:
summary(fit)

**More recent data**

Here is a data set collected by Loren Collingwood, a political scientist. It accumulates many of the same county-level variables but is looking at the recent national election instead. Let's try out our Rpart skills!

In [ ]:
election = read.csv("http://www.collingwoodresearch.com/uploads/8/3/6/0/8360930/county_data.csv",sep="\t")
head(election)

Make a column indicating the winner for each county.

In [ ]:
election = mutate(election,winner = ifelse(pct_clinton < pct_trump,"Trump","Clinton"))
head(election)

In [ ]:
fit = rpart(winner~per_capita_income+pobama12cnty+percent_white,data=election)

In [ ]:
par(xpd=TRUE)

plot(fit)
text(fit,use.n=TRUE,cex=0.8)

Collingwood didn't have a dictionary for his data, but he was able to answer a few questions. Here are the columns I couldn't read easily and his explanations.

totpop1014cnty what does 1014 refer to? 1014 IS SAMPLED BETWEEN YEARS 2010-14 FOR ACES
<br>
ppiwht1014cnty   ppi? --I'M PRETTY SURE THIS IS PERCENT WHITE, THN PERCENT BLACK, ETC.
<br>
ppiblk1014cnty
<br>
ppihisp1014cnty
<br>
pimm1014cnty        mm? PERCENT IMMIGRANT
<br>
ppi0014cnty         ppi again and what is 0014? THE 0014 WOULD NOW BE A CHANGE MEASURE -- PERCENT CHANGE FROM 2000 TO 2014.
<br>
psocsec0014cnty
<br>
ownthirty0014cnty   ownthirty?
<br>
ownfifty0014cnty
<br>
pmanufact1014cnty   ? employment in manufacturing? CORRECT -- THIS WAS PREDICTOR OF TRUMP VOTE
<br>
pct65over00cnty     this is in 2000? YES, FOR OLD PEOPLE
<br>
medoohvalue00cnty    ? MEDIAN HOME VALUE 2000
<br>
pctcomlong00cnty    comlong? I THINK THIS IS COMMUTE OVER 30 MINS -- BUT I DIDN'T USE THIS SO NOT 100% ON THAT.
<br>
pctdiffh95_00cnty   ? NOT SURE
<br>
pctfb00cnty         ? FOREIGN BORN
<br>
pcpi00cnty           pcpi? NOT SURE
<br>
Rpt.                ? NOT SURE
<br>
state.y STATE NAME/CODE. PROBABLY A state.x AS WELL? FUNCTION OF MERGING IN R
<br>
pct_change_R         ?
<br>
manu_loss            ? IS THIS A DUMMY? IF SO, DID THE COUNTY RECEIVE MANUFACTURING LOSS IF YES = 1, ELSE = 0
<br>
rural                ? RURAL COUNTY BASED OFF OFFICIAL CENSUS DESIGNATION (DUMMY)
<br>
rural_150            ? SIMILAR MEASURE, THIS IS THE ONE I THINK IS BASED OF 100 PER SQ/MILE. BUT WOULD HAVE TO CHECK